In [20]:
# Importing the required classes and libraries
from itertools import islice, chain
import pandas as pd
import nltk

In [21]:
def get_index_positions(list_of_elems, element):
  ''' Returns the indices of all occurrences of given element in
  the list- list_of_elems '''
  index_pos_list = []
  index_pos = 0
  while True:
    try:
      # Search for item in list from index_pos to the end of list
      index_pos = list_of_elems.index(element, index_pos)
      # Add the index position in list
      index_pos_list.append(index_pos)
      index_pos += 1
    except ValueError:
      break
  return index_pos_list

In [22]:
def convert_list_1dto2d(list_of_elems, list_of_lengths): 
  ''' Returns a 2-dimensional list of given variable lengths-
  list_of_lengths from the 1-dimensional list- list_of_elems '''
  # Generate an iterator of the 1-d list
  iterator = iter(list_of_elems)
  # Slice the 1-d list in variable lengths and add it to a 2-d list
  return [list(islice(iterator, i)) for i in list_of_lengths]

In [23]:
def preprocess_dataset(filename):
  ''' Return the preprocessed dataset from the dataset- filename
  in  the format of a 2-d list of sentences with each sentence being
  a list of tuples- (word, tag) '''
  sentences = []
  try:
    with open(filename) as f:
      for line in f:
        # Read each line in the file
        sentences.append(tuple(word for word in line.split()[:-1]))
  except IOError:
    print('There is no file named: ', filename)       
  # Add ('##', '##') and ('&&', '&&') to mark the beginning and end, respectively, of each sentence    
  for index, phrase in enumerate(sentences):
    if phrase == ('.', '.'):
      # Insert ('&&', '&&') after each ('.', '.')
      sentences.insert(index + 1, ('&&', '&&'))
    elif phrase == ():
      sentences[index] = ('##', '##')
  # Append ('##', '##') to the beginning of the first sentence
  sentences.insert(0, ('##', '##'))
  # Get indices of all occurrences of ('&&','&&')
  indices = get_index_positions(sentences, ('&&','&&'))
  # Generate the lengths of each sentence in the dataset
  length_first_sentence = indices[0]
  length_of_sentences = [indices[i + 1] - indices[i] for i in range(len(indices)-1)]
  length_of_sentences = [length_first_sentence + 1] + length_of_sentences
  # Create a 2-d list of sentences
  # eg. [sentence1, sentence2, ...]
  sentences = convert_list_1dto2d(sentences, length_of_sentences)
  return sentences

In [24]:
training_data_sentences = preprocess_dataset('train.txt')

In [25]:
# Print the first 5 sentences in the preprocessed training dataset
training_data_sentences[0:5]

[[('##', '##'),
  ('Confidence', 'NN'),
  ('in', 'IN'),
  ('the', 'DT'),
  ('pound', 'NN'),
  ('is', 'VBZ'),
  ('widely', 'RB'),
  ('expected', 'VBN'),
  ('to', 'TO'),
  ('take', 'VB'),
  ('another', 'DT'),
  ('sharp', 'JJ'),
  ('dive', 'NN'),
  ('if', 'IN'),
  ('trade', 'NN'),
  ('figures', 'NNS'),
  ('for', 'IN'),
  ('September', 'NNP'),
  (',', ','),
  ('due', 'JJ'),
  ('for', 'IN'),
  ('release', 'NN'),
  ('tomorrow', 'NN'),
  (',', ','),
  ('fail', 'VB'),
  ('to', 'TO'),
  ('show', 'VB'),
  ('a', 'DT'),
  ('substantial', 'JJ'),
  ('improvement', 'NN'),
  ('from', 'IN'),
  ('July', 'NNP'),
  ('and', 'CC'),
  ('August', 'NNP'),
  ("'s", 'POS'),
  ('near-record', 'JJ'),
  ('deficits', 'NNS'),
  ('.', '.'),
  ('&&', '&&')],
 [('##', '##'),
  ('Chancellor', 'NNP'),
  ('of', 'IN'),
  ('the', 'DT'),
  ('Exchequer', 'NNP'),
  ('Nigel', 'NNP'),
  ('Lawson', 'NNP'),
  ("'s", 'POS'),
  ('restated', 'VBN'),
  ('commitment', 'NN'),
  ('to', 'TO'),
  ('a', 'DT'),
  ('firm', 'NN'),
  ('monetary'

In [26]:
testing_data_sentences  = preprocess_dataset('test.txt')

In [27]:
# Print the first 5 sentences in the preprocessed testing dataset
testing_data_sentences[0:5]

[[('##', '##'),
  ('Rockwell', 'NNP'),
  ('International', 'NNP'),
  ('Corp.', 'NNP'),
  ("'s", 'POS'),
  ('Tulsa', 'NNP'),
  ('unit', 'NN'),
  ('said', 'VBD'),
  ('it', 'PRP'),
  ('signed', 'VBD'),
  ('a', 'DT'),
  ('tentative', 'JJ'),
  ('agreement', 'NN'),
  ('extending', 'VBG'),
  ('its', 'PRP$'),
  ('contract', 'NN'),
  ('with', 'IN'),
  ('Boeing', 'NNP'),
  ('Co.', 'NNP'),
  ('to', 'TO'),
  ('provide', 'VB'),
  ('structural', 'JJ'),
  ('parts', 'NNS'),
  ('for', 'IN'),
  ('Boeing', 'NNP'),
  ("'s", 'POS'),
  ('747', 'CD'),
  ('jetliners', 'NNS'),
  ('.', '.'),
  ('&&', '&&')],
 [('##', '##'),
  ('Rockwell', 'NNP'),
  ('said', 'VBD'),
  ('the', 'DT'),
  ('agreement', 'NN'),
  ('calls', 'VBZ'),
  ('for', 'IN'),
  ('it', 'PRP'),
  ('to', 'TO'),
  ('supply', 'VB'),
  ('200', 'CD'),
  ('additional', 'JJ'),
  ('so-called', 'JJ'),
  ('shipsets', 'NNS'),
  ('for', 'IN'),
  ('the', 'DT'),
  ('planes', 'NNS'),
  ('.', '.'),
  ('&&', '&&')],
 [('##', '##'),
  ('These', 'DT'),
  ('include', 

In [28]:
# Generate a dictionary of tags with its corresponding word and word count
# eg. {tag: {word1: count(word1, tag), word2: count(word2, tag), ...}, ...}
train_tag_dictionary = {}
for sentence in training_data_sentences:
  for (word, tag) in sentence:
    word = word.lower()
    try:
      try:
        # Subsequent appearances of word for the tag
        train_tag_dictionary[tag][word] += 1
      except:
        # First appearance of word for the tag
        train_tag_dictionary[tag][word] = 1
    except:
      # Introducing the word for the tag
      train_tag_dictionary[tag] = {word:1}

In [29]:
# Calculate the emission probabilities using dictionary of trained data-
# train_tag_dictionary for each tag
train_emission_probabilities = {}
for tag in train_tag_dictionary.keys():
  train_emission_probabilities[tag] = {}
  # Total count of the words for the tag
  count = sum(train_tag_dictionary[tag].values())
  for word in train_tag_dictionary[tag].keys():
    # Divide word-count by total count for the tag
    train_emission_probabilities[tag][word] = train_tag_dictionary[tag][word]/count

In [30]:
# Determine the bigrams for the transition probabilities by the bigram tag data
# dictionary- tag_bigrams
tag_bigrams = {}
for sentence in training_data_sentences:
  bigram = list(nltk.bigrams(sentence))
  for tag1, tag2 in bigram:
    try:
      try:
        # Subsequent appearances of tag2 for the tag1
        tag_bigrams[tag1[1]][tag2[1]] += 1
      except:
        # First appearance of tag2 for the tag1
        tag_bigrams[tag1[1]][tag2[1]] = 1
    except:
      # Introducing the tag2 for the tag1
      tag_bigrams[tag1[1]] = {tag2[1]:1}

In [31]:
# Calculate the tag bigrams' probabilities for transition probabilities  
tag_bigram_probabilities = {}
for tag1 in tag_bigrams.keys():
  tag_bigram_probabilities[tag1] = {}
  # Total count of the tag2 for the tag1
  count = sum(tag_bigrams[tag1].values())
  for tag2 in tag_bigrams[tag1].keys():
    # Divide tag2 count by total count for the tag1
    tag_bigram_probabilities[tag1][tag2] = tag_bigrams[tag1][tag2]/count

In [32]:
# Calculate possible tags for each of the word in training and test
# dataset (because some words maybe unique to test data)
def possible_token_tags(data_sentences):
  ''' Return all the possoble tags for each token in the preprocessed
  dataset- data_sentences '''
  token_tags = {}
  for sentence in data_sentences:
    for (word, tag) in sentence:
      # Change case to lower for consistency
      word = word.lower()
      try:
        # First appearance of tag for the word
        if tag not in token_tags[word]:
          token_tags[word].append(tag)
      except:
        # Subsequent appearances of tag for the word
        rem_tag = []
        rem_tag.append(tag)
        token_tags[word] = rem_tag
  return token_tags

token_tags = {}
# Detemine the possible tags of each word in training and testing dataset
token_tags = possible_token_tags(training_data_sentences + testing_data_sentences)

In [33]:
# Split the testing dataset into words and tags
test_words = []
test_tags = []
# For each sentence in testing dataset
for sentence in testing_data_sentences:
  temp_word = []
  temp_tag = []
  for (word, tag) in sentence:
    temp_word.append(word.lower())
    temp_tag.append(tag)
  # List of words in testing dataset
  test_words.append(temp_word)
  # List of corresponding tags in the testing dataset
  test_tags.append(temp_tag)

In [34]:
# The Viterbi Algorithm
predicted_tags = []
for sentence in test_words:
  # viterbi stores the required intermediate values 
  # eg. viterbi = {state_num : {state1: [previous_best_state, value_of_state], ...}}                
  viterbi = {}
  for index, state_word in enumerate(sentence):
    # For the first word in the sentence
    if index == 1:
      viterbi[index] = {}
      tags = token_tags[state_word]
      for tag in tags:
        try:
          # When the word is present in the training set
          viterbi[index][tag] = ['##', tag_bigram_probabilities['##'][tag]*train_emission_probabilities[tag][state]]
        except:
          # When the  word is unique to testing test i.e. not present in the training set,
          # assign a probability of 0.0001
          viterbi[index][tag] = ['##', 0.0001]
    # For the words other than the first word
    if index > 1:
      viterbi[index] = {}
      # All the previous states
      previous_states = list(viterbi[index - 1].keys())
      # All the current states
      current_states  = token_tags[state_word]
      # Calculate the maximum previous viterbi value from the previous states to each
      # current states
      for cs in current_states:
        temp = []
        for ps in previous_states:
          try:
            # When the word is present in the training set
            temp.append(viterbi[index - 1][ps][1]*tag_bigram_probabilities[ps][cs]*train_emission_probabilities[cs][state_word])
          except:
            # When the  word is unique to testing test i.e. not present in the training set,
            # assign a probability of 0.0001
            temp.append(viterbi[index - 1][ps][1]*0.0001)
        # Maximum value for the best previous state
        max_temp_index = temp.index(max(temp))
        best_ps = previous_states[max_temp_index]
        viterbi[index][cs]=[best_ps, max(temp)]
  # Backtrack to extract the best possible tags for each word
  pred_tags = []
  # Total states in the algoritm
  total_states = viterbi.keys()
  last_state_num = max(total_states)
  for bs in range(len(total_states)):
    state_num = last_state_num - bs
    if state_num == last_state_num:
      # For the end of the sentence
      pred_tags.append('&&')
      pred_tags.append(viterbi[state_num]['&&'][0])
    if state_num < last_state_num and state_num > 0:
      # Other words in the sentence from the viterbi dictionary
      pred_tags.append(viterbi[state_num][pred_tags[len(pred_tags) - 1]][0])
  # Reverse the list- pred_tags for correct tags
  predicted_tags.append(list(reversed(pred_tags)))

In [35]:
# Create iterable chains from the 2-d lists- test_words, test_tags, predicted_tags
separated_test_words = list(chain.from_iterable(test_words))
separated_test_tags = list(chain.from_iterable(test_tags))
separated_predicted_tags = list(chain.from_iterable(predicted_tags))
# Table of word, its corresponding tag and the predicted tag
table = []
for test_word, test_tag, predicted_tag in zip(separated_test_words, separated_test_tags, separated_predicted_tags):
  table.append([test_word, test_tag, predicted_tag])
# Dataframe for the table to display given tags and predicted tags
dataframe = pd.DataFrame(table, columns=['Word', 'Given Tag', 'Predicted Tag'])

In [36]:
# Print the word, its corresponding tag and the predicted tag
dataframe

,Word,Given Tag,Predicted Tag
0,##,##,##
1,rockwell,NNP,NNP
2,international,NNP,NNP
3,corp.,NNP,NNP
4,'s,POS,POS
...,...,...,...
51336,to,TO,TO
51337,mr.,NNP,NNP
51338,harlow,NNP,NNP
51339,.,.,.


In [37]:
# Calculate the accuracy from the predicted tags
true_prediction_count = 0 
false_prediction_count = 0
for index, test_sentence in enumerate(test_tags):
  predicted = predicted_tags[index]
  for inner_index, given in enumerate(test_sentence):
    # Increment if given tag and predicted tag match
    if given == predicted[inner_index]:
      true_prediction_count += 1
    else:
      false_prediction_count += 1
# Accuracy and Loss calculations
accuracy = (true_prediction_count/(true_prediction_count+false_prediction_count))*100
loss = (false_prediction_count/(true_prediction_count+false_prediction_count))*100
print('Accuracy: {0:.2f}%'.format(accuracy))
print('Loss: {0:.2f}%'.format(loss))

Accuracy: 96.53%
Loss: 3.47%
